In [3]:
import json
import pandas as pd
import mysql.connector

In [12]:
USER = "root"
SENHA = "ZAQwsx123!@#"
HOST = "localhost"
DATABASE = "datatran"
try:
    cnx = mysql.connector.connect(user=USER,
                              password=SENHA,
                              host=HOST,
                              database=DATABASE)
    print("Conexão estabelecida")
except mysql.connector.Error as e: 
    print("Conexão não estabelecida", e)

Conexão não estabelecida 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


In [ ]:
cnx.close()

In [ ]:
data = []

estados = list(df['uf'].unique())
estados.sort()

for estado in estados:
  data.append({"estado" : estado,
   "Acidentes" : 0,
   "Fatalidades" : 0,
   "Via Mais Perigosa" : "****",
   "Principal Causa" : "****"
  })

In [ ]:
with open("saida/dados.json", "w") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)